#Streamlit


In [2]:
# !pip -q install streamlit

     |████████████████████████████████| 9.2 MB 8.1 MB/s 
     |████████████████████████████████| 237 kB 57.2 MB/s 
     |████████████████████████████████| 4.7 MB 47.6 MB/s 
     |████████████████████████████████| 164 kB 60.0 MB/s 
     |████████████████████████████████| 182 kB 60.3 MB/s 
     |████████████████████████████████| 78 kB 8.7 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 51 kB 8.0 MB/s 


In [3]:
#%%writefile app.py
from datetime import date
from dateutil import relativedelta
import datetime
import pandas as pd



def request_search(bearer_token, params, max_count):

    tweets = []
    expanded = {
        "tweets": [],
        "users": []
    }

    next_token = None

    while True:
        if next_token is not None:
            params["next_token"] = next_token

        url = "https://api.twitter.com/2/tweets/search/recent"
        encoded_params = urllib.parse.urlencode(params)
        headers = {"Authorization": f"Bearer AAAAAAAAAAAAAAAAAAAAAJ5bZwEAAAAAdlSGg9zHc8pH2IKr5HgBllUg3SA%3D3ZpLpfDZHf4Sd01v4np7rJcr3DyEXBfNTWBfpOGST30OtqGGdG"}
        res = requests.request(
            "GET", url, params=encoded_params, headers=headers)

        if res.status_code == 429:
            rate_limit_reset = int(res.headers["x-rate-limit-reset"])
            now = time.mktime(datetime.datetime.now().timetuple())
            wait_sec = int(rate_limit_reset - now)
            desc = f"Waiting for {wait_sec} seconds"
            for _ in tqdm.trange(wait_sec, desc=desc):
                time.sleep(1)

        elif res.status_code != 200:
            raise Exception(res.status_code, res.text)

        else:
            res_json = res.json()

            if res_json["meta"]["result_count"] == 0:
                break

            tweets += res_json["data"]
            print(f"{len(tweets)}件のツイートを取得しました。")

            if res_json.get("includes"):
                includes = res_json["includes"]
                for k, v in expanded.items():
                    if includes.get(k):
                        
                        expanded[k] += includes[k]

            next_token = res_json.get("meta").get("next_token")
            if next_token is None or len(tweets) >= max_count:
                break

    return tweets[:max_count], expanded


def export_json(fpath, data):
    with open(fpath, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False)


def search_tweet(max_count,keyword,value):

  dt_now_jst_aware = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
  print((dt_now_jst_aware - relativedelta.relativedelta(hours=value)).isoformat())
  print((dt_now_jst_aware - relativedelta.relativedelta(minutes=1)).isoformat())


  bearer_token = "AAAAAAAAAAAAAAAAAAAAAJ5bZwEAAAAAdlSGg9zHc8pH2IKr5HgBllUg3SA%3D3ZpLpfDZHf4Sd01v4np7rJcr3DyEXBfNTWBfpOGST30OtqGGdG"
  max_count = max_count

  params = {
      "query":keyword + str('-RT'),
      "start_time": (dt_now_jst_aware - relativedelta.relativedelta(hours=value)).isoformat(),
      "end_time": (dt_now_jst_aware - relativedelta.relativedelta(minutes=1)).isoformat(),
      "expansions": "author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id",
      "max_results": "100",
      "media.fields": "duration_ms,height,media_key,preview_image_url,type,url,width,public_metrics",
      "place.fields": "contained_within,country,country_code,full_name,geo,id,name,place_type",
      "poll.fields": "duration_minutes,end_datetime,id,options,voting_status",
      "tweet.fields": "attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,public_metrics,possibly_sensitive,referenced_tweets,reply_settings,source,text,withheld",
      "user.fields": "created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld"
  }


  tweets, expanded = request_search(bearer_token, params, max_count)


  df_tweet = pd.DataFrame()
  df_tweet_ex = pd.DataFrame()
  for i in range(0,len(tweets)):
      try:
          #df_tweet.loc[i,'created_at'] = pd.to_datetime(tweets[i]['created_at']) + datetime.timedelta(hours=9)
          df_tweet.loc[i,'author_id'] = tweets[i]['author_id']
          df_tweet.loc[i,'text'] = tweets[i]['text']
          #df_tweet.loc[i,'tweet_id'] = tweets[i]['id']
          df_tweet.loc[i,'like_count'] = tweets[i]['public_metrics']['like_count']
          df_tweet.loc[i,'retweet_count'] = tweets[i]['public_metrics']['retweet_count']
          #df_tweet.loc[i,'reply_count'] = tweets[i]['public_metrics']['reply_count']

          #df_tweet_ex.loc[i,'regist_date'] = pd.to_datetime(expanded['users'][i]['created_at']) + datetime.timedelta(hours=9)
          df_tweet_ex.loc[i,'name'] = expanded['users'][i]['name']
          df_tweet_ex.loc[i,'username'] = expanded['users'][i]['username']
          df_tweet_ex.loc[i,'description'] = expanded['users'][i]['description']
          #df_tweet_ex.loc[i,'following_count'] = expanded['users'][i]['public_metrics']['following_count']
          #df_tweet_ex.loc[i,'followers_count'] = expanded['users'][i]['public_metrics']['followers_count']
          #df_tweet_ex.loc[i,'listed_count'] = expanded['users'][i]['public_metrics']['listed_count']
          #df_tweet_ex.loc[i,'tweet_count'] = expanded['users'][i]['public_metrics']['tweet_count']
          #df_tweet_ex.loc[i,'prof_link'] = expanded['users'][i]['url']
          #df_tweet_ex.loc[i,'pinned_tweet_id'] = expanded['users'][i]['pinned_tweet_id']
          df_tweet_ex.loc[i,'author_id'] = expanded['users'][i]['id']

      except:
          pass


  df_tweet = df_tweet.merge(df_tweet_ex,on='author_id',how='inner')
  df_tweet['url'] = df_tweet['username'].apply(lambda x: 'https://twitter.com/' + str(x))
  df_tweet = df_tweet[['name','username','text','description','retweet_count','like_count','url']]
  return df_tweet

def make_clickable(val): 
  return '<a href="{}">{}</a>'.format(val,val)


#---------- Web apps ----------#
import streamlit as st
import base64


pagelist = ["TPA","Analytics"]

st.set_page_config(layout="wide")
selector=st.sidebar.selectbox( "Mode",pagelist)
if selector=="TPA":
  st.title("Talentum：Talent Pool Automation")
  cnt=st.number_input('探索ツイート数の設定：0~50000',0,50000,0,step=1)
  keyword = st.text_input('人材探索キーワードの設定 半角で入力ください')
  st.text_area('分析メモ')


  talent_search = st.button("Search Talent")
  if talent_search :
    df_tweet = search_tweet(cnt,keyword,24*6.9)
    df_talent = df_tweet[['username','text','description','url']].rename(columns={'username':'ユーザーID','text':'ツイート本文','description':'プロフィール','url':'ツイートのURL'})
    df_talent = df_talent.groupby('ツイート本文',as_index=False).head(1)
  df_talent


#  description = st.text_input('プロフに含まれるキーワードの設定 （正規表現）')
#  profile_search = st.button("check profile")
#  if profile_search :
#    df_display = df_talent[df_talent['description'].str.contains('description')]
#  df_display


  csv = df_talent.to_csv(index=False)
  b64 = base64.b64encode(csv.encode()).decode()
  href = f'<a href="data:application/octet-stream;base64,{b64}" download="result_utf-8.csv">Download Link</a>'
  st.markdown(f"人材探索データのダウンロード（csv）:  {href}", unsafe_allow_html=True)

elif st.button('Analytics'):
  st.title("Coming soon")



Overwriting app.py


In [4]:
# from google.colab import files
# files.view("/content")
# files.view("app.py")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
# !streamlit run app.py & sleep 3 && npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.11.24:8501

npx: installed 22 in 4.136s
your url is: https://nice-onions-repeat-34-125-11-24.loca.lt
2022-12-13 12:47:00.086 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 564, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 152, in <module>
    df_talent
NameError: name 'df_talent' is not defined
2022-12-13 12:47:06.439 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 564, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 152, in <module>
    df_talent
NameError: name 'df_talent' is not defined
  Stopping...
^C
